In [1]:
import glob
import os
import json
import numpy as np
import scipy.linalg as linalg
import pandas as pd
import mne
import matplotlib.pyplot as plt
import seaborn as sb
from shutil import copyfile
from tqdm import tqdm
from data_util import (
    load_subjects,
    load_subject_data,
    load_data,
    euclidean_alignment,
    label_alignment,
)  # download from https://gist.github.com/txdat/e6583def88d95b4ded049703e2ac3cc2

In [2]:
%%time
# subjects = [
#     'K299', 'K300', 'K309', 'K310', 'K311',
#     'K312', 'K313', 'K314', 'K315', 'K316',
#     'K318', 'K319', 'K320', 'K321', 'K322',
#     'K323', 'K324', 'K325', 'K326', 'K327',
#     'K328', 'K329', 'K330', 'K331', 'K332',
#     'K333', 'K334', 'K335', 'K336', 'K337',
#     'K338', 'K339', 'K340', 'K342', 'K343',
#     'K344', 'K350', 'K351', 'K352', 'K353',
#     'K354', 'K355', 'K356', 'K357', 'K358',
#     'K359', 'K360', 'K361', 'K362', 'K363',
#     'K364', 'K365', 'K366', 'K367', 'K368',
#     'K369', 'K370', 'K371', 'K372', 'K373',
#     'K374', 'K375',
# ]

subjects = [
    "K300",
    "K309",
    "K311",
    "K312",
    "K313",
    "K315",
    "K316",
    "K321",
    "K322",
    "K323",
    "K324",
    "K326",
    "K328",
    "K329",
    "K337",
    "K338",
    "K339",
    "K343",
    "K355",
    "K358",
    "K359",
    "K360",
    "K361",
    "K365",
    "K368",
    "K369",
    "K370",
    "K371",
    "K372",
    "K375",
]

# use_k_subjects = True  # @param {"type": "boolean"}
# use_bn_subjects = False  # @param {"type": "boolean"}

# subjects = list()
# if use_k_subjects:
#     subjects.extend([subject for subject in full_scenarios_subjects if subject.startswith("K")])
# if use_bn_subjects:
#     subjects.extend([subject for subject in full_scenarios_subjects if subject.startswith("BN")])

subjects = sorted(subjects)

scenarios = [
    "nâng tay trái",
    "nâng tay phải",
    "nâng chân trái",
    "nâng chân phải",
    "gật đầu",
    "lắc đầu",
    # "há miệng",
]

events = [
    "Thinking",
    # "Thinking and Acting",
    # "Resting",
    # "Typing",
]

# channels = [
#     "Fp1", "F7", "F3", "FC5", "FC1", "T7", "C3", "CP5", "CP1", "P7", "P3", "O1",  # left
#     "Fz", "Cz", "Pz", "Oz",  # center
#     "Fp2", "F8", "F4", "FC6", "FC2", "T8", "C4", "CP6", "CP2", "P8", "P4", "O2",  # right
# ]
channels = [
    "Fp1",
    "Fp2",
    "F7",
    "F3",
    "Fz",
    "F4",
    "F8",
    "FC5",
    "FC1",
    "FC2",
    "FC6",
    "T7",
    "C3",
    "Cz",
    "C4",
    "T8",
    "CP5",
    "CP1",
    "CP2",
    "CP6",
    "P7",
    "P3",
    "Pz",
    "P4",
    "P8",
    "O1",
    "Oz",
    "O2",
]

label_mapping = {
    "nâng chân trái_Thinking": "nâng chân_Thinking",
    "nâng chân phải_Thinking": "nâng chân_Thinking",
    "gật đầu_Thinking": "đầu_Thinking",
    "lắc đầu_Thinking": "đầu_Thinking",
}

n_channels = len(channels)

print(f"using {n_channels} channels")

minimal_trial_duration = 4  # @param
window_duration = 2  # @param
window_stride_duration = 0.25  # @param

fmin = 4.0  # @param
fmax = 30.0  # @param

ds = load_data(
    "../data/vin/Official",
    subjects=subjects,
    scenarios=scenarios,
    events=events,
    channels=channels,
    label_mapping=label_mapping,
    minimal_trial_duration=minimal_trial_duration,
    window_duration=window_duration,
    window_stride_duration=window_stride_duration,
    start_offset=0,
    stop_offset=0,
    fmin=fmin,
    fmax=fmax,
    resample=None,
    return_raw=False,
    preload=True,
)

subject:   0%|                                                             | 0/94 [00:00<?, ?it/s]

using 28 channels


subject:  44%|██████████████████████▋                             | 41/94 [00:01<00:03, 14.26it/s]/home/txdat/code/eeg-motor-imagery/notebooks/data_util.py:201: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(f"{sample_dir}/EEG.edf", preload=False, verbose=False)
subject:  82%|██████████████████████████████████████████▌         | 77/94 [00:02<00:00, 21.35it/s]/home/txdat/code/eeg-motor-imagery/notebooks/data_util.py:201: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(f"{sample_dir}/EEG.edf", preload=False, verbose=False)
/home/txdat/code/eeg-motor-imagery/notebooks/data_util.py:201: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = mne.io.read_raw_edf(f"{sample_dir}/EEG.edf", preload=False, verbose=False)
/home/txdat/code/eeg-motor-imagery/notebooks/data_util.py:201: RuntimeWarning: Limited 1 annotation(s) that were e

CPU times: user 21.8 s, sys: 3.26 s, total: 25 s
Wall time: 31.9 s


In [4]:
from typing import Optional
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as td

# from pytorch_metric_learning.losses import ArcFaceLoss
from braindecode.util import set_random_seeds
from braindecode.models import ShallowFBCSPNet, Deep4Net, EEGNetv4
from braindecode import EEGClassifier
from braindecode.training.losses import CroppedLoss
from skorch.callbacks import (
    LRScheduler,
    EarlyStopping,
    Initializer,
    LoadInitState,
    Freezer,
    Checkpoint,
    TrainEndCheckpoint,
)
from skorch.helper import predefined_split
from skorch.setter import optimizer_setter
from skorch.dataset import unpack_data
from skorch.utils import to_tensor
from sklearn.utils.class_weight import compute_class_weight
import sklearn.metrics as metrics
from functools import partial
from copy import deepcopy
from tqdm import tqdm

In [17]:
class ImbalancedDataSampler(td.Sampler):
    def __init__(self, targets: np.ndarray):  # [len(data),]
        self.num_samples = len(targets)

        count = np.bincount(targets)
        self.weight = torch.tensor(1.0 / count[targets]).float()

    def __len__(self):
        return self.num_samples

    def __iter__(self):
        return (
            i
            for i in torch.multinomial(self.weight, self.num_samples, replacement=True)
        )

In [21]:
ds.datasets[0].y

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [18]:
targets = list()
for epochs in ds.datasets:
    targets.extend(epochs.y)

targets = np.array(targets, dtype=int)

loader = td.DataLoader(
    ds, batch_size=16, sampler=ImbalancedDataSampler(targets=targets)
)

In [19]:
for x, y, _ in loader:
    print(x.size(), y.size())
    break

TypeError: iteration over a 0-d tensor